# Agenda

- Generators (generator functions)
- Concurrency
    - Threads
    - Processes
    - `asyncio`
- Profiling    
- NumPy + Pandas

- Single-quoted string: `''`
- Double-quoted string: `""`
- `"""triple quoted string"""`
- `r'c:\a\b\c\d\e'`  # auto-doubles backslashes
- `b'abc'` # bytes, not characters
- `f'abc{x}'`  # replaces anything in `{}` with its value -- starting from Python 3.6

In [2]:
# 3.8 added trailing = to f-strings

x = 100
y = [10, 20, 30]

f'{x=}, {y=}'

'x=100, y=[10, 20, 30]'